In [146]:
import pandas as pd
import numpy as np

## Задание 0


Для всех  заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **tr_mcc_codes, tr_types, transactions и gender_train** из одноимённых таблиц из папки [data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). 

Для таблицы transactions используйте только первые n=1000000 строк. При чтении файлов обратите внимание на разделители внутри каждого из файлов - они могут различаться!

###  Описание данных
#### Таблица ```transactions.csv```
##### Описание
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### Формат данных

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### Описание полей

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком; ```+``` — начисление средств клиенту (приходная транзакция), ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала;


#### Таблица ```gender_train.csv```

##### Описание
Данная таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### Формат данных
```
customer_id,gender
111111,0
111112,1
...
```

##### Описание полей
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента;

### Таблица ```tr_mcc_codes.csv```

##### Описание
Данная таблица содержит описание mcc-кодов транзакций.

##### Формат данных
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### Описание полей
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

#### Таблица ```tr_types.csv```

##### Описание
Данная таблица содержит описание типов транзакций.

##### Формат данных
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### Описание полей
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции;

In [147]:
df_tr_mcc_codes = pd.read_csv('./sources/tr_mcc_codes.csv', sep=';')
df_tr_types = pd.read_csv('./sources/tr_types.csv', sep=';')
df_transactions = pd.read_csv('./sources/transactions.csv', sep=',')
df_gender_train = pd.read_csv('./sources/gender_train.csv', sep=',')


Задания 1-4 делаются без использования merge!

## Задание 1



1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM'


Выбранные строки типов транзакций

In [148]:
import re
df_selected_tr_types = pd.DataFrame(df_transactions['tr_type'].sample(n=1000, random_state=2));
df_selected_tr_types

,tr_type
1520974,7070
1908930,2010
4168903,2011
1122190,2330
5863833,7011
...,...
5740230,2010
2693560,1030
6818935,1210
2094828,2011


Ключ / расшифровка транзакции

In [149]:
df_sorted_tr_key_value = df_tr_types.sort_values(by = ['tr_type'])
df_sorted_tr_key_value

,tr_type,tr_description
97,1000,Покупка.ТУ СБ РФ
98,1010,Покупка. POS ТУ СБ РФ
99,1030,Оплата услуги. Банкоматы СБ РФ
100,1100,Покупка. ТУ Россия
101,1110,Покупка. POS ТУ Россия
...,...,...
79,8151,Урегулирование безнадежной задолженности
80,8210,Плата за приост. действия карты
81,8220,Плата за обработку счета банковской карты клие...
82,8230,Плата за предоставление по запросу клиента док...


Массив выбранных типов

In [150]:

list_tr_types = list(set(list(df_selected_tr_types['tr_type'])))
list_tr_types.sort()
list_tr_types


# df_tr_types['tr_type'].apply(lambda x: x if x in list_tr_types else 'bruh')




[1010,
 1030,
 1100,
 1110,
 1200,
 1210,
 2010,
 2011,
 2020,
 2100,
 2110,
 2330,
 2331,
 2370,
 2371,
 2440,
 4010,
 4011,
 4051,
 4071,
 4097,
 4210,
 6100,
 6110,
 7010,
 7011,
 7020,
 7021,
 7030,
 7031,
 7041,
 7070,
 7071]

Получаем датафрейм выбранных типов транзакций и их расшифровок

In [151]:
list_exist_in_selected_tr_types = list(df_sorted_tr_key_value['tr_type'].apply(lambda x: x in list_tr_types))
df_filtered_by_tr_type_selection = df_sorted_tr_key_value.loc[list_exist_in_selected_tr_types];
df_filtered_by_tr_type_selection

,tr_type,tr_description
98,1010,Покупка. POS ТУ СБ РФ
99,1030,Оплата услуги. Банкоматы СБ РФ
100,1100,Покупка. ТУ Россия
101,1110,Покупка. POS ТУ Россия
102,1200,Покупка. Зарубеж. ТУ
103,1210,Покупка. POS Зарубеж. ТУ
106,2010,Выдача наличных в АТМ Сбербанк России
107,2011,Выдача наличных в АТМ Сбербанка (в других ТБ)
108,2020,Выдача наличных через POS Сбербанка
110,2100,Наличные. Россия


Получение описаний транзакций с наличием 'POS' и 'ATM'

In [152]:
import re
df_filtered_by_has_words = df_filtered_by_tr_type_selection.loc[df_filtered_by_tr_type_selection['tr_description'].apply(lambda x: bool(re.search("POS | АТМ", x)))]
df_filtered_by_has_words


,tr_type,tr_description
98,1010,Покупка. POS ТУ СБ РФ
101,1110,Покупка. POS ТУ Россия
103,1210,Покупка. POS Зарубеж. ТУ
106,2010,Выдача наличных в АТМ Сбербанк России
107,2011,Выдача наличных в АТМ Сбербанка (в других ТБ)
108,2020,Выдача наличных через POS Сбербанка
111,2110,Наличные. АТМ Россия
118,2330,Списание с карты по операции “перевода с карты...
119,2331,Списание с карты по операции “перевода с карты...
5,4010,Плата за получение наличных в АТМ Сбербанка


Доля найденных строк к выбранному числу транзакций (n / 1000)

In [159]:
df_filtered_by_has_words.shape[0] / 1000

0.019

## Задание 2 


1. Для столбца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Выведите топ-10 транзакций по частоте встречаемости (вывести для них tr_description тоже).

## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту
2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте
3. Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов

## Задание 4
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций из топ 10 из задания 2
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций для клиентов из задания 3

## Подготовка для заданий 5-8

*Из заданий 5-8 нужно выполнить минимум два любых*

Соедините transactions с всеми остальными таблицами (tr_mcc_codes, tr_types, gender_train). Причём с gender_train необходимо смёрджиться с помощью left join, а с оставшимися датафреймами - через inner.
После получения результата таблицы gender_train, tr_types, tr_mcc_codes можно удалить. В результате соединения датафреймов должно получиться 999584 строки.

In [154]:
transactions = pd.merge(transactions, gender_train, how='left')
transactions = pd.merge(transactions, tr_mcc_codes, how='inner')
transactions = pd.merge(transactions, tr_types, how='inner')
transactions.shape

NameError: name 'transactions' is not defined

## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин

Обратите внимание, что для вычисления модуля разности точных знаний о том, 
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

## Задание 6

1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 типов транзакций для мужчин и для женщин, наименьших среди всех типов транзакций по полученным значениям "max_income". 
2. Выделите среди них те типы транзакций, которые встречаются одновременно и у мужчин, и у женщин

## Задание 7

1. Найдите суммы затрат по каждой категории (mcc) для мужчин и для женщин. 
2. Найдите топ 10 категорий с самыми большими относительными модулями разности в тратах для разных полов (в ответе должны присутствовать описания mcc кодов).

## Задание 8

1. Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":". (\**)
2. Посчитайте количество транзакций с amount<0 в ночное время для мужчин и женщин. Ночное время - это примерно 00-06 часов. 